[View in Colaboratory](https://colab.research.google.com/github/Ankita-Das/Movie_Recommender/blob/master/movierec.ipynb)

In [2]:
from google.colab import files
uploaded=files.upload()


Saving u.data to u.data
Saving u.item to u.item


In [9]:
import numpy as np
import pandas as pd
header=['user_id','item_id','rating','timestamp']
df=pd.read_csv('u.data',sep='\t',names=header)
print(df)

       user_id  item_id  rating  timestamp
0          196      242       3  881250949
1          186      302       3  891717742
2           22      377       1  878887116
3          244       51       2  880606923
4          166      346       1  886397596
5          298      474       4  884182806
6          115      265       2  881171488
7          253      465       5  891628467
8          305      451       3  886324817
9            6       86       3  883603013
10          62      257       2  879372434
11         286     1014       5  879781125
12         200      222       5  876042340
13         210       40       3  891035994
14         224       29       3  888104457
15         303      785       3  879485318
16         122      387       5  879270459
17         194      274       2  879539794
18         291     1042       4  874834944
19         234     1184       2  892079237
20         119      392       4  886176814
21         167      486       4  892738452
22         

In [10]:
n_users=df.user_id.unique().shape[0]
n_items=df.item_id.unique().shape[0]
print('Name of users='+str(n_users)+'| Number of movies='+ str(n_items))
train_data=df

Name of users=943| Number of movies=1682


In [11]:
#creating user-item matrices for training and testing
train_data_matrix=np.zeros((n_users,n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1,line[2]-1]=line[3]
print(train_data_matrix)


[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]


In [16]:
#@title Default title text
from sklearn.metrics.pairwise import pairwise_distances
item_similarity=1-pairwise_distances(train_data_matrix.T,metric='cosine')
print(item_similarity)
item_similarity=item_similarity.argsort()
    


[[1.         0.40238218 0.33024479 ... 0.         0.04718307 0.04718307]
 [0.40238218 1.         0.27306918 ... 0.         0.07829936 0.07829936]
 [0.33024479 0.27306918 1.         ... 0.         0.         0.09687505]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.04718307 0.07829936 0.         ... 0.         1.         0.        ]
 [0.04718307 0.07829936 0.09687505 ... 0.         0.         1.        ]]


In [19]:
def input_user():
    n=int(input('Enter the no. of recommendations you want:'))
    mv=int(input('Enter the movie id of the movie you liked:'))
    print('Movies similar to the ones you liked:')
    mv=mv+1
    header=['movie id','movie title','release date','video release date','IMDb URL','unknown','Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
    df2=pd.read_csv('u.item',sep='|',names=header,encoding='latin-1')
    '''The movie titles are not present in the dataset u.data,it is being taken from another dataset by matching the movie ids'''
    rec_items=np.array((item_similarity[[mv],:n]))
    import matplotlib.pyplot as plt
    for mr in np.nditer(rec_items):
        print(mr)
        for index,rows in df2.iterrows():
            if(index==mr):
                print(rows['movie title'])
    return
input_user()


Enter the no. of recommendations you want:10
Enter the movie id of the movie you liked:40
Movies similar to the ones you liked:
1454
Outlaw, The (1943)
1506
Three Lives and Only One Death (1996)
1504
Killer: A Journal of Murder (1995)
1324
August (1996)
1328
Low Life, The (1994)
1233
Chairman of the Board (1998)
710
Substance of Fire, The (1996)
599
Daniel Defoe's Robinson Crusoe (1996)
1322
Wooden Man's Bride, The (Wu Kui) (1994)
598
Police Story 4: Project S (Chao ji ji hua) (1993)


In [18]:
#########################
#This part of the program is for prediction of ratings given the user id and the movie id.
#i.e.prediction : if the user would have seen that movie what rating he/she would have given!!!

user_similarity=1-pairwise_distances(train_data_matrix,metric='cosine')
def predict(ratings,similarity,type='user'):
    if type =='user':
        mean_user_rating=ratings.mean(axis=1)
        ratings_diff=(ratings - mean_user_rating[:,np.newaxis])
        pred=mean_user_rating[:,np.newaxis]+similarity.dot(ratings_diff)/np.array([np.abs(similarity).sum(axis=1)]).T
        print('User-Item similarity wise predicted ratings for each user to each movie:')
        print(pred)
    elif type=='item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
        print('Item-Item similarity wise predicted ratings for each user to each movie:')
        print(pred)
    return pred
item_prediction = predict(train_data_matrix, item_similarity, type='item')
print('\n\n')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

Item-Item similarity wise predicted ratings for each user to each movie:
[[1.00196856 0.93090079 0.93717926 ... 0.21260631 0.20788614 0.0932822 ]
 [0.26046511 0.22301572 0.22916332 ... 0.04971278 0.04978847 0.03989189]
 [0.17633465 0.14585339 0.14760975 ... 0.04641651 0.04502303 0.03404703]
 ...
 [0.09821598 0.08627516 0.09023067 ... 0.01162393 0.01454177 0.02210549]
 [0.36669682 0.32659414 0.3221456  ... 0.07818162 0.07964796 0.10013857]
 [0.59013695 0.5524902  0.54353582 ... 0.11169955 0.12481459 0.14276296]]



User-Item similarity wise predicted ratings for each user to each movie:
[[ 2.69523211  1.02855273  0.71474438 ...  0.27084883  0.27682149
   0.27767246]
 [ 1.81499156  0.25796184  0.23493818 ... -0.0641761  -0.06477416
  -0.06563205]
 [ 1.2342326   0.19665872  0.14191646 ... -0.08590333 -0.09164035
  -0.09307737]
 ...
 [ 2.28350935  0.27870273  0.22722929 ... -0.16848565 -0.165704
  -0.16668048]
 [ 1.96457986  0.51224185  0.22792838 ... -0.07327901 -0.07002751
  -0.07204701]